In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('D:\\Документы\\Programs\\DataFile\\movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1417,tt0380389,30000000,27610873,Goal!: The Dream Begins,Kuno Becker|Alessandro Nivola|Anna Friel|Steph...,Danny Cannon,Every Dream Has A Beginning,"Like millions of kids around the world, Santia...",118,Drama,Touchstone Pictures|Milkshake Films|Hargitay &...,6/6/2005,6.5,2005
1718,tt0211933,86000000,14373825,Awake,Hayden Christensen|Jessica Alba|Terrence Howar...,Joby Harold,"Every year, one in 700 people wake up during s...","While undergoing heart surgery, a man experien...",84,Thriller|Crime|Mystery,The Weinstein Company|Open City Films|GreeneSt...,11/28/2007,6.3,2007
1818,tt0187078,90000000,237202299,Gone In Sixty Seconds,Nicolas Cage|Angelina Jolie|Giovanni Ribisi|De...,Dominic Sena,"Ice Cold, Hot Wired.",Upon learning that he has to come out of retir...,118,Action|Crime|Thriller,Jerry Bruckheimer Films|Touchstone Pictures,6/9/2000,6.0,2000
381,tt0435761,200000000,1063171911,Toy Story 3,Tom Hanks|Tim Allen|Ned Beatty|Joan Cusack|Mic...,Lee Unkrich,No toy gets left behind.,"Woody, Buzz, and the rest of Andy's toys haven...",103,Animation|Family|Comedy,Walt Disney Pictures|Pixar Animation Studios,6/16/2010,7.5,2010
1699,tt0486576,130000000,289047763,4: Rise of the Silver Surfer,Ioan Gruffudd|Jessica Alba|Chris Evans|Michael...,Tim Story,Discover the secret of the Surfer.,The Fantastic Four return to the big screen as...,92,Adventure|Fantasy|Action|Thriller,Ingenious Film Partners|Twentieth Century Fox ...,6/13/2007,5.4,2007


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

In [5]:
data['profit'] = data.revenue - data.budget # прибыль = сборы - бюджет

In [6]:
data['month'] = data.release_date.apply(lambda x: x.split('/')[0]) # месяц из полной даты 

In [37]:
# разделение строки из колонки genres на несколько строк новой таблицы с учетом разделителя между названиями жанров одного фильма
# в функцию передается ID фильма, по которому выбираются его жанры, заданных через разделитель
# переданные ID и жанры по нему сохраняются в новый глобальный датафрэйм (таблицу) ganre_spl
def genre_split(id):
    global genre_spl # итоговая таблица
    
    temp = pd.DataFrame(columns=['imdb_id', 'genres']) # временная таблица
    cur = data[['genres']][data.imdb_id == id] # таблица с одной строкой фильма и его жанрами, найденными по ID
    list_c = cur.iloc[0].genres.split('|') # список всех жанров одного фильма
    temp.genres = list_c # сохраняем спискок в новую временную таблицу
    temp.imdb_id = id # ID фильма общий
    
    # записываем жанры одного фильма в общую таблицу, созданную вне этой функции
    genre_spl = pd.concat([genre_spl, temp])
    return

In [43]:
# разделение строки из колонки director на несколько строк новой таблицы с учетом разделителя между режиссерами одного фильма
# в функцию передается ID фильма, по которому выбираются его режиссеры, заданные через разделитель, и кассовые сборы
# переданные ID, режиссеры и сборы по нему сохраняются в новый глобальный датафрэйм (таблицу) director_spl
def director_split(id):
    global director_spl # итоговая таблица
    
    temp = pd.DataFrame(columns=['imdb_id', 'director', 'revenue']) # временная таблица
    cur = data[['director', 'revenue']][data.imdb_id == id] # таблица с одной строкой фильма, его режиссерами и сборами, найденными по ID
    list_c = cur.iloc[0].director.split('|') # список всех режиссеров одного фильма
    temp.director = list_c # сохраняем спискок в новую временную таблицу
    temp.imdb_id = id # ID фильма общий
    temp.revenue = int(cur.iloc[0].revenue/len(list_c)) # сбор фильма
    #temp.revenue = int(cur.iloc[0].revenue/len(list_c)) # сбор делиться на количесвто режиссеров в фильме
    
    # записываем режиссеров одного фильма в общую таблицу, созданную вне этой функции
    director_spl = pd.concat([director_spl, temp])
    return

In [9]:
# разделение строки из колонки cast на несколько строк новой таблицы с учетом разделителя между актрами одного фильма
# в функцию передается ID фильма, по которому выбираются его актеры, заданные через разделитель, и кассовые сборы
# переданные ID, актеры и сборы по нему сохраняются в новый глобальный датафрэйм (таблицу) cast_spl
def cast_split(id):
    global cast_spl # итоговая таблица
    
    temp = pd.DataFrame(columns=['imdb_id', 'cast', 'revenue']) # временная таблица
    cur = data[['cast', 'revenue']][data.imdb_id == id] # таблица с одной строкой фильма и его актерами и сборами, найденными по ID
    list_c = cur.iloc[0].cast.split('|') # список всех актеров одного фильма
    temp.cast = list_c # сохраняем спискок в новую временную таблицу
    temp.imdb_id = id # ID фильма общий
    temp.revenue = cur.iloc[0].revenue # общий сбор от фильма
    
    # записываем режиссеров одного фильма в общую таблицу, созданную вне этой функции
    cast_spl = pd.concat([cast_spl, temp])
    return

In [10]:
# разделение строки из колонки production_companies на несколько строк новой таблицы с учетом разделителя между студиями одного фильма
# в функцию передается ID фильма, по которому выбираются его студии, заданные через разделитель, название фильма и описание
# переданные ID, студия, название фильма, длина названия, описание и длина описания по фильму
# сохраняются в новый глобальный датафрэйм (таблицу) companies_spl
def companies_split(id):
    global companies_spl # итоговая таблица
    
    temp = pd.DataFrame(columns=['imdb_id', 'companies', 'title', 'len_title', 'overview', 'len_overview']) # временная таблица
    cur = data[['production_companies', 'original_title', 'overview']][data.imdb_id == id] # таблица с одной строкой фильма, его студиями, названием и описанием фильма, найденными по ID
    list_c = cur.iloc[0].production_companies.split('|') # список всех студий одного фильма
    temp.companies = list_c # сохраняем спискок в новую временную таблицу
    temp.imdb_id = id # ID фильма общий
    temp.title = cur.iloc[0].original_title # название фильма
    temp.len_title = len(cur.iloc[0].original_title) # длина назв.
    temp.overview = cur.iloc[0].overview # описание фильма
    temp.len_overview = len(cur.iloc[0].overview.split(' ')) # кол-во слов описания
    
    # записываем студии одного фильма в общую таблицу, созданную вне этой функции
    companies_spl = pd.concat([companies_spl, temp])
    return

In [11]:
# разделение строки из колонки cast на несколько строк новой таблицы с учетом разделителя между актрами одного фильма
# в функцию передается ID фильма, по которому выбираются его актеры, заданные через разделитель
# актеры одного фильма разбиваются на все возможные пары и сохраняются в каждой строке по парно
# переданные ID и актеры по нему сохраняются в новый глобальный датафрэйм (таблицу) cast_pair
def cast_split_pair(id):
    global cast_pair #итоговая таблица
    temp = pd.DataFrame(columns=['imdb_id', 'cast_pair']) # временная таблица
    cur = data[['imdb_id','cast']][data.imdb_id == id] # таблица с одной строкой фильма и его актерами, найденными по ID
    list_c = cur.iloc[0].cast.split('|') # список всех актеров одного фильма
    temp_pair_film = [] # временный список для пар артистов одного фильма
    for x in range(0, len(list_c)-2): # каждого артиста соединяем со всеми последующими в списке
        for y in range(x+1, len(list_c)-1): # следующие артисты
            temp_pair = [] # временный список для каждой пары артистов
            temp_pair.append(list_c[x])
            temp_pair.append(list_c[y])
            #display(temp_pair_l)
            temp_pair_film.append(temp_pair)
            #display(temp_pair_film)
                                 
    temp.cast_pair = temp_pair_film # сохраняем спискок в новую временную таблицу
    temp.imdb_id = id # ID фильма общий
    
    # записываем пары актеров одного фильма в общую таблицу, созданную вне этой функции
    cast_pair = pd.concat([cast_pair, temp])
    return

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [12]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
#answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)' #+
# если ответили верно, можете добавить комментарий со значком "+"

In [13]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011,641683000,5


ВАРИАНТ 2

In [14]:
# можно добавлять разные варианты решения
data[['original_title', 'imdb_id']][data.budget == data.budget.max()]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


# 2. Какой из фильмов самый длительный (в минутах)?

In [16]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)' #+

In [15]:
data[['original_title', 'imdb_id']][data.runtime == data.runtime.max()]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [17]:
data[['original_title', 'imdb_id']][data.runtime == data.runtime.min()]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


In [18]:
answers['3'] = '3. Winnie the Pooh (tt1449283)' #+

# 4. Какова средняя длительность фильмов?


In [19]:
round(data.runtime.mean(),0)

110.0

In [20]:
answers['4'] = '2. 110' #+

# 5. Каково медианное значение длительности фильмов? 

In [21]:
data.runtime.median()

107.0

In [22]:
answers['5'] = '1. 107' #+

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [23]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[['original_title', 'imdb_id']][data.profit == data.profit.max()]

,original_title,imdb_id
239,Avatar,tt0499549


In [24]:
answers['6'] = '5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [25]:
data[['original_title', 'imdb_id']][data.profit == data.profit.min()]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [26]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [27]:
data.imdb_id[data.profit > 0].count()

1478

In [28]:
answers['8'] = '1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [29]:
d2008 = data[data.release_year == 2008]
d2008[['original_title', 'imdb_id']][d2008.revenue == d2008.revenue.max()]

,original_title,imdb_id
599,The Dark Knight,tt0468569


In [30]:
answers['9'] = '4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [31]:
d12_14 = data.query('2012 <= release_year <= 2014')
d12_14[['original_title', 'imdb_id']][d12_14.profit == d12_14.profit.min()]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [32]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [38]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
genre_spl = pd.DataFrame(columns=['imdb_id', 'genres']) # список фильмов с жанрами по одному на каждый фильм

data.imdb_id.apply(genre_split)
genre_spl = genre_spl.reset_index()
genre_spl = genre_spl.drop(['index'], axis = 1)
genre_spl.genres.value_counts().index[0]

'Drama'

ВАРИАНТ 2

In [39]:
allfilms_genres = pd.DataFrame(columns=['genres']) # итоговая таблица для записи каждого жанра в отдельную строку по всем фильмам 
for genres in data.genres:
    cur = pd.DataFrame(columns=['genres']) # временная таблица для записи всех жанров одного фильма построчно
    cur.genres = genres.split('|') # разделяем каждый жанр в отдельную строку таблицы 
    allfilms_genres = pd.concat([allfilms_genres, cur]) # добавляем жанры одного фильма в итоговую табл 
allfilms_genres.genres.value_counts().index[0] # получаем первую строку с наибольшим значением в серии из жанров

'Drama'

In [79]:
answers['11'] = '3. Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [40]:
genre_profit = genre_spl.merge(data, on='imdb_id', how='left')
genre_profit[genre_profit.profit > 0].genres_x.value_counts().index[0]

'Drama'

In [41]:
answers['12'] = '1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [44]:
director_spl = pd.DataFrame(columns=['imdb_id', 'director', 'revenue']) # список фильмов с режиссерами по одному на каждый фильм

data.imdb_id.apply(director_split)
director_spl = director_spl.reset_index()
director_spl = director_spl.drop(['index'], axis = 1)
director_spl.groupby(['director'])['revenue'].sum().sort_values(ascending=False).head()

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
Name: revenue, dtype: int64

In [45]:
answers['13'] = '5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [46]:
director_genre = director_spl.merge(genre_spl, on='imdb_id', how='left')
director_genre[director_genre.genres == 'Action'].director.value_counts().index[0]

'Robert Rodriguez'

In [47]:
answers['14'] = '3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [48]:
cast_spl = pd.DataFrame(columns=['imdb_id', 'cast', 'revenue']) # список фильмов с режиссерами по одному на каждый фильм

data.imdb_id.apply(cast_split)
cast_spl = cast_spl.reset_index()
cast_spl = cast_spl.drop(['index'], axis = 1)

cast_profit = cast_spl.merge(data, on='imdb_id', how='left')
cast_profit[cast_profit['release_year'] == 2012].groupby(['cast_x'])['revenue_x'].sum().sort_values(ascending=False).head()

cast_x
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
Name: revenue_x, dtype: int64

In [49]:
answers['15'] = '3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [50]:
cast_profit[cast_profit['revenue_x'] > data['revenue'].mean()].cast_x.value_counts()

Tom Cruise        14
Ben Stiller       14
Adam Sandler      14
Cameron Diaz      14
Angelina Jolie    13
                  ..
Rory Kinnear       1
Josh Helman        1
Aksel Hennie       1
Matt Walsh         1
Eric Winter        1
Name: cast_x, Length: 1355, dtype: int64

In [52]:
answers['16'] = '1. Tom Cruise'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [53]:
cast_genre = cast_spl.merge(genre_spl, on='imdb_id', how='left')
cast_genre[cast_genre['cast'] == 'Nicolas Cage'].genres.value_counts().index[0]

'Action'

In [54]:
answers['17'] = '2. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [55]:
data[data['production_companies'].str.contains('Paramount Pictures')]\
    [['imdb_id', 'original_title', 'profit']].sort_values('profit')

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034
1123,tt0300556,Timeline,-60519261
1722,tt0435705,Next,-51788987
1623,tt0375173,Alfie,-46604061
1631,tt0315297,Twisted,-24805000
...,...,...,...
730,tt1229238,Mission: Impossible - Ghost Protocol,549713380
1738,tt0418279,Transformers,559709780
603,tt0367882,Indiana Jones and the Kingdom of the Crystal S...,601636033
317,tt1055369,Transformers: Revenge of the Fallen,686297228


In [56]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [57]:
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).index[0]

2015

In [58]:
answers['19'] = '5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [59]:
data[data['production_companies'].str.contains('Warner Bros')]\
    .groupby(['release_year'])['profit'].sum().sort_values(ascending=False).index[0]

2014

In [60]:
answers['20'] = '1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [63]:
data.month.value_counts().index[0]

'9'

In [64]:
answers['21'] = '4. Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [65]:
data[data['month'].isin(['6','7','8'])]['imdb_id'].count()

450

In [66]:
answers['22'] = '2. 450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [67]:
director_data = director_spl.merge(data, on='imdb_id', how='left')
director_data[director_data['month'].isin(['1','11','12'])].director_x.value_counts().index[0]

'Peter Jackson'

In [68]:
answers['23'] = '5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [69]:
companies_spl = pd.DataFrame(columns=['imdb_id', 'companies', 'title', 'len_title', 'overview', 'len_overview']) # список фильмов со студиями по одному на каждый фильм

data.imdb_id.apply(companies_split)
companies_spl = companies_spl.reset_index()
companies_spl = companies_spl.drop(['index'], axis = 1)
companies_spl.len_title = companies_spl.len_title.apply(lambda x: int(x))
companies_spl.len_overview = companies_spl.len_overview.apply(lambda x: int(x))

companies_spl[['companies', 'title', 'len_title']].sort_values('len_title', ascending=False).head()

,companies,title,len_title
4687,Four By Two Productions,Borat: Cultural Learnings of America for Make ...,83
4686,Twentieth Century Fox Film Corporation,Borat: Cultural Learnings of America for Make ...,83
4231,Walden Media,"The Chronicles of Narnia: The Lion, the Witch ...",62
4230,Walt Disney,"The Chronicles of Narnia: The Lion, the Witch ...",62
750,Walt Disney Pictures,"Alexander and the Terrible, Horrible, No Good,...",59


In [70]:
answers['24'] = '5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [71]:
companies_spl.groupby(['companies'])['len_overview'].mean().sort_values(ascending=False).head()

companies
Midnight Picture Show               175.0
Room 9 Entertainment                161.0
Heineken Branded Entertainment      161.0
98 MPH Productions                  159.0
Brookwell-McNamara Entertainment    156.0
Name: len_overview, dtype: float64

In [72]:
answers['25'] = '3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [73]:
# выводим 1% (1/100) всех фильмов отсоритрованных по убыванию рейтинга
data[['original_title', 'vote_average']].sort_values(['vote_average'], ascending=False).head(int(data.vote_average.count()/100))

,original_title,vote_average
599,The Dark Knight,8.1
118,Interstellar,8.0
125,The Imitation Game,8.0
9,Inside Out,8.0
34,Room,8.0
1183,The Wolf of Wall Street,7.9
128,Gone Girl,7.9
1191,12 Years a Slave,7.9
119,Guardians of the Galaxy,7.9
1081,The Lord of the Rings: The Return of the King,7.9


In [74]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [75]:
cast_pair = pd.DataFrame(columns=['imdb_id', 'cast_pair']) # список фильмов с парами актеров на каждый фильм

data.imdb_id.apply(cast_split_pair)
cast_pair = cast_pair.reset_index()
cast_pair = cast_pair.drop(['index'], axis = 1)

cast_pair.cast_pair.value_counts()

[Daniel Radcliffe, Emma Watson]       8
[Daniel Radcliffe, Rupert Grint]      8
[Rupert Grint, Emma Watson]           7
[Robert Pattinson, Taylor Lautner]    5
[Kristen Stewart, Taylor Lautner]     5
                                     ..
[Chris Evans, Anna Faris]             1
[Joshua Jackson, Lena Headey]         1
[Miley Cyrus, Jaden Smith]            1
[Jim Belushi, Janeane Garofalo]       1
[Amanda Seyfried, Jason Sudeikis]     1
Name: cast_pair, Length: 10855, dtype: int64

ВАРИАНТ 2

In [82]:
data_c = pd.DataFrame(columns=['cast'])
data_c.cast = data.cast
data_c['cast'] = data_c.cast.str.split('|')
data_p = data_c['cast'].reset_index()
display(data_p.head())

from itertools import combinations
data_p['pairs'] = data_p['cast'].apply(lambda s: list(itertools.combinations(s, 2)))
data_p = data_p.explode('pairs').reset_index()
collections.Counter(data_p['pairs']).most_common(1)

,index,cast
0,0,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan..."
1,1,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,..."
2,2,"[Shailene Woodley, Theo James, Kate Winslet, A..."
3,3,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad..."
4,4,"[Vin Diesel, Paul Walker, Jason Statham, Miche..."


NameError: name 'itertools' is not defined

In [77]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

# Submission

In [80]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '1. Tom Cruise',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint',
 '11': '3. Drama'}

In [81]:
# и убедиться что ни чего не пропустил)
len(answers)

27